<a href="https://colab.research.google.com/github/smruthyunjaya05/MLE/blob/main/Noise_Aware_Circuit_Compilation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install qiskit

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit.transpiler import Layout
from qiskit import QuantumCircuit, transpile

# ============================================================
# CONFIGURATION
# ============================================================

NUM_QUBITS = 64
EPISODES = 2000
EPSILON_START = 0.5
EPSILON_DECAY = 0.995
RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)

# ============================================================
# LOAD SYNTHETIC BACKEND
# ============================================================

backend = GenericBackendV2(num_qubits=NUM_QUBITS, seed=RANDOM_SEED)
target = backend.target
coupling_map = backend.coupling_map

try:
    edges = list(coupling_map.get_edges())
except:
    try:
        edges = list(coupling_map.edges)
    except:
        g = nx.random_regular_graph(d=3, n=NUM_QUBITS, seed=RANDOM_SEED)
        edges = list(g.edges())

# ============================================================
# MULTI-ARMED BANDIT AGENT (NOISE-AWARE)
# ============================================================

class DeepNoiseBandit:
    def __init__(self, n_qubits, epsilon=EPSILON_START):
        self.n_qubits = n_qubits
        self.q_table = np.zeros(n_qubits, dtype=float)
        self.counts = np.zeros(n_qubits, dtype=int)
        self.epsilon = epsilon

    def select_qubit(self):
        if np.random.rand() < self.epsilon:
            return np.random.randint(self.n_qubits)
        return int(np.argmax(self.q_table))

    def update(self, qubit, reward):
        self.counts[qubit] += 1
        alpha = 1.0 / (self.counts[qubit] + 1)
        self.q_table[qubit] += alpha * (reward - self.q_table[qubit])

    def decay_epsilon(self):
        self.epsilon *= EPSILON_DECAY

# ============================================================
# HARDWARE-INSPIRED REWARD FUNCTION
# ============================================================

def get_physics_reward(qubit_idx):
    readout_err = 0.1
    try:
        readout_meta = target.get("measure", None)
        if readout_meta:
            readout_err = readout_meta.get((qubit_idx,), readout_err).error
        else:
            readout_err = float(np.clip(np.random.normal(0.03, 0.02), 0.001, 0.2))
    except:
        readout_err = float(np.clip(np.random.normal(0.03, 0.02), 0.001, 0.2))

    t1 = np.random.uniform(50e-6, 150e-6)
    t2 = np.random.uniform(20e-6, 100e-6)
    coherence_score = (t1 + t2) * 1e5
    error_penalty = (readout_err * 100) + 1.0

    return coherence_score / error_penalty

# ============================================================
# TRAINING LOOP
# ============================================================

agent = DeepNoiseBandit(NUM_QUBITS)
history = []

for episode in range(EPISODES):
    q = agent.select_qubit()
    reward = get_physics_reward(q)
    agent.update(q, reward)
    agent.decay_epsilon()
    history.append(reward)

# ============================================================
# NORMALIZE Q-TABLE
# ============================================================

qvals = agent.q_table.astype(float)
qmin, qmax = qvals.min(), qvals.max()

if np.isclose(qmax, qmin):
    norm_q = np.zeros_like(qvals)
else:
    norm_q = (qvals - qmin) / (qmax - qmin)

# ============================================================
# PLOT 1: LEARNING CURVE
# ============================================================

smooth = np.convolve(history, np.ones(50) / 50, mode='valid')

plt.figure(figsize=(10, 4))
plt.plot(smooth)
plt.title("Noise-Aware Agent Learning Curve")
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# ============================================================
# PLOT 2: FINAL Q-TABLE DISTRIBUTION
# ============================================================

plt.figure(figsize=(10, 4))
plt.bar(range(NUM_QUBITS), agent.q_table)
plt.title("Final Learned Qubit Quality Scores")
plt.xlabel("Qubit Index")
plt.ylabel("Score")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# ============================================================
# COLOR MAP FOR 3D VISUALIZATION
# ============================================================

def viridis_rgb(t):
    if t <= 0.5:
        a = t / 0.5
        r = int((68 * (1-a) + 32 * a))
        g = int((1 * (1-a) + 144 * a))
        b = int((84 * (1-a) + 140 * a))
    else:
        a = (t - 0.5) / 0.5
        r = int((32 * (1-a) + 253 * a))
        g = int((144 * (1-a) + 231 * a))
        b = int((140 * (1-a) + 37 * a))
    return f'rgb({r},{g},{b})'

colors_rgb = [viridis_rgb(float(x)) for x in norm_q]

# ============================================================
# 3D SPHERE LAYOUT FOR QUBITS
# ============================================================

def fibonacci_sphere(n, radius=1.0):
    pts = []
    phi = np.pi * (3. - np.sqrt(5.))
    for i in range(n):
        y = 1 - (i / float(n - 1)) * 2
        radius_proj = np.sqrt(1 - y * y)
        theta = phi * i
        x = np.cos(theta) * radius_proj
        z = np.sin(theta) * radius_proj
        pts.append((x * radius, y * radius, z * radius))
    return pts

coords = fibonacci_sphere(NUM_QUBITS)
xs, ys, zs = zip(*coords)

edge_x, edge_y, edge_z = [], [], []
for (u, v) in edges:
    if u < NUM_QUBITS and v < NUM_QUBITS:
        x0, y0, z0 = coords[u]
        x1, y1, z1 = coords[v]
        edge_x += [x0, x1, None]
        edge_y += [y0, y1, None]
        edge_z += [z0, z1, None]

node_trace = go.Scatter3d(
    x=xs, y=ys, z=zs,
    mode='markers+text',
    marker=dict(size=6, color=colors_rgb),
    text=[str(i) for i in range(NUM_QUBITS)],
    hovertext=[f"Q{i}: score={agent.q_table[i]:.3f}, norm={norm_q[i]:.3f}"
               for i in range(NUM_QUBITS)],
    hoverinfo="text"
)

edge_trace = go.Scatter3d(
    x=edge_x, y=edge_y, z=edge_z,
    mode='lines',
    line=dict(width=1, color='rgba(100,100,100,0.4)')
)

fig = go.Figure(data=[edge_trace, node_trace])
fig.update_layout(
    title=f"3D Qubit Preference Globe ({NUM_QUBITS} qubits)",
    scene=dict(xaxis=dict(visible=False),
               yaxis=dict(visible=False),
               zaxis=dict(visible=False))
)
fig.show()

# ============================================================
# OPTION A — NOISE-AWARE CIRCUIT → PHYSICAL MAPPING
# ============================================================

def noise_aware_layout(circuit, qtable):
    """
    Maps logical qubits of the circuit onto the best physical qubits.
    """
    num_logical = circuit.num_qubits
    ranked = np.argsort(qtable)[::-1]  # best → worst
    selected = ranked[:num_logical]

    layout_dict = {
        circuit.qubits[i]: int(selected[i])
        for i in range(num_logical)
    }

    return Layout(layout_dict)

# ------------------------------------------------------------
# Example circuit to test the noise-aware compilation
# ------------------------------------------------------------

qc = QuantumCircuit(3)
qc.h(0); qc.h(1); qc.h(2)
qc.x(0); qc.x(1)
qc.h(2)
qc.ccx(0, 1, 2)
qc.h(2)
qc.x(0); qc.x(1)


print("\nOriginal Circuit:")
print(qc)

layout = noise_aware_layout(qc, agent.q_table)
print("\nNoise-Aware Initial Layout:")
print(layout)

optimized = transpile(
    qc,
    backend=backend,
    initial_layout=layout,
    routing_method="sabre"
)

print("\n=== Noise-Aware Transpiled Circuit ===")
print(optimized)

# ============================================================
# SUMMARY OUTPUTS
# ============================================================

best_q = int(np.argmax(agent.q_table))
worst_q = int(np.argmin(agent.q_table))

print("\n--- ANALYSIS REPORT ---")
print(f"Best Physical Qubit: #{best_q}")
print(f"Worst Physical Qubit: #{worst_q}")
print("Noise-aware mapping completed successfully.")


In [ ]:
"""
1. Basic H–X–CX Chain
# Simple 3-qubit chain: prepare each qubit, then entangle sequentially
c1 = QuantumCircuit(3)
c1.h(0); c1.x(0)
c1.h(1); c1.x(1)
c1.h(2); c1.x(2)
c1.cx(0,1)    # first entanglement
c1.cx(1,2)    # second entanglement

2. Star Topology (Routing Stress Test)
# Central qubit entangles with all others -- stresses layout/routing
c2 = QuantumCircuit(4)
c2.h(0)
c2.cx(0,1)
c2.cx(0,2)
c2.cx(0,3)

3. Dense Clifford Circuit
# Mixture of Clifford gates + cross qubit CNOT interactions
c3 = QuantumCircuit(4)
c3.h(0); c3.s(1); c3.x(2); c3.z(3)
c3.cx(0,2)
c3.cx(1,3)
c3.h(2)

4. Toffoli (CCX) Gate Test
# CCX gate to test multi-qubit decomposition on real hardware
c4 = QuantumCircuit(3)
c4.x(0); c4.x(1)
c4.ccx(0,1,2)

5. Alternating Entanglement Pattern
# Mix of entanglements across non-adjacent qubits
c5 = QuantumCircuit(5)
for i in range(5):
    c5.h(i)
c5.cx(0,4)
c5.cx(1,3)
c5.cx(2,0)

6. Phase Estimation Sub-Block
# Controlled-phase chain + final entangling CNOT
c6 = QuantumCircuit(3)
c6.h(0)
c6.cp(np.pi/4, 0, 1)
c6.cp(np.pi/8, 0, 2)
c6.cx(1,2)

7. 5-Qubit Quantum Fourier Transform Core
# Full QFT pattern: Hadamard then decreasing controlled phases
c7 = QuantumCircuit(5)
for i in range(5):
    c7.h(i)
    for j in range(i+1, 5):
        c7.cp(np.pi/(2**(j-i)), i, j)

8. Randomized Gate Layer
# Random Clifford mix -- good for mapping diversity
c8 = QuantumCircuit(4)
c8.h(0); c8.y(1); c8.z(2); c8.x(3)
c8.cx(3,2)
c8.h(1)
c8.z(2)

9. Noisy Bell State Variant
# Bell pair but with extra gates inserted
c9 = QuantumCircuit(2)
c9.h(0)
c9.cx(0,1)
c9.x(0)
c9.h(1)
c9.z(0)

10. Ladder Entanglement
# Linear chain entanglement, ideal for heavy routing tests
c10 = QuantumCircuit(6)
c10.h(0)
for i in range(5):
    c10.cx(i, i+1)

11. Repeated Forward/Backward CNOT Layers
# Stress-test circuit: many CNOTs both directions
c11 = QuantumCircuit(5)
c11.h(range(5))
for i in range(4):
    c11.cx(i, i+1)
for i in reversed(range(4)):
    c11.cx(i+1, i)

12. GHZ State Generator
# Standard GHZ: 000 → 000 + 111
c12 = QuantumCircuit(4)
c12.h(0)
c12.cx(0,1)
c12.cx(1,2)
c12.cx(2,3)

13. Grover Oracle Block
# 3-qubit Grover oracle + diffusion
c13 = QuantumCircuit(3)
c13.h([0,1,2])
c13.ccx(0,1,2)
c13.h([0,1,2])

14. Teleportation Protocol (No Measurements)
# Teleportation entangling portion only
c14 = QuantumCircuit(3)
c14.h(1)
c14.cx(1,2)
c14.cx(0,1)
c14.h(0)

15. Mini “Quantum Convolution” Kernel
# Two input qubits feeding into one output qubit via CNOTs
c15 = QuantumCircuit(3)
c15.h(0); c15.h(1)
c15.cx(0,2)
c15.cx(1,2)
c15.z(2)
"""
